# [Flight API](https://www.flightapi.io/docs/) Data Pipeline
The purpose of this notebook is show my steps and thought process of building a pipeline from an inital REST API Call.

## Purpose and Goals of Project
Overall, flight information across various sources is constantly scattered from a range of different airports, airlines, and locations. The goal and value of this project is to pull data from different sources and organize such data sources that can then be viewed directly or later used as source of visualization. With this, a user can clearly see insightful flight information across differing sources in an organized fashion.

### Notes and Possible Areas of Improvement : 
After completing this project, there are a couple things that can be improved upon to increase efficiency and further promote better standards which I will technically address in the future. First, since this notebook does not contain any form of orchestration I am actively learning and improving my skills in Databricks to implement orchestration tools and other languages that could be of use like Pyspark, Scala or SQL. Additionally, while starting this project I was quite unfamilarly with working with json files of such a complex form (as shown in the variable dict1). Because of this, I decided to work with the key of most significance, define a function which flattens data and then create a pandas dataframe from such data. Although this method works well in this case, in the future I plan on instead working directly with the json file, ridding unneccessary information from there and then creating the dataframe in order to follow best practice standards, promoting efficiency. 

In [86]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [87]:
# import
import requests
import pandas as pd
import json
import datetime
import pytz

## API Call and Initial Exploration

In [88]:
# Perform API call and output response code, '200' meaning a success
url = "https://api.flightapi.io/schedule/63e3f3ebc750d7e9767923c2?mode=departures&day=1&iata=DOH"
response = requests.get(url)
print(response.status_code)

200


In [89]:
# Assign to dictionary and reduce documentation
dict1=response.json()
dict1=dict1['airport']['pluginData']
dict1

{'details': {'name': 'Doha Hamad International Airport',
  'code': {'iata': 'DOH', 'icao': 'OTHH'},
  'delayIndex': {'arrivals': 0.42, 'departures': 1.73},
  'stats': {'arrivals': {'delayIndex': 0.42,
    'delayAvg': 5,
    'percentage': {'delayed': 0, 'canceled': 0, 'trend': 'down'},
    'recent': {'delayIndex': 0.42,
     'delayAvg': 5,
     'percentage': {'delayed': 0, 'canceled': 0, 'trend': 'down'},
     'quantity': {'onTime': 9, 'delayed': 0, 'canceled': 0}},
    'today': {'percentage': {'delayed': 0.01, 'canceled': 0.01},
     'quantity': {'onTime': 26, 'delayed': 3, 'canceled': 2}},
    'yesterday': {'percentage': {'delayed': 0.16, 'canceled': 0.01},
     'quantity': {'onTime': 248, 'delayed': 47, 'canceled': 2}},
    'tomorrow': {'percentage': {'canceled': 0.01},
     'quantity': {'canceled': 4}}},
   'departures': {'delayIndex': 1.73,
    'delayAvg': 20,
    'percentage': {'delayed': 0.75, 'canceled': 0.01, 'trend': 'down'},
    'recent': {'delayIndex': 1.73,
     'delayAvg':

In [90]:
dict1.keys()

dict_keys(['details', 'flightdiary', 'schedule', 'weather', 'aircraftCount', 'runways'])

In [91]:
# Search for useful data
dict1['runways']

[{'name': '16L',
  'length': {'ft': 15912, 'm': 4850},
  'surface': {'code': 'ASPH', 'name': 'Asphalt'}},
 {'name': '16R',
  'length': {'ft': 13944, 'm': 4250},
  'surface': {'code': 'ASPH', 'name': 'Asphalt'}},
 {'name': '34L',
  'length': {'ft': 13944, 'm': 4250},
  'surface': {'code': 'ASPH', 'name': 'Asphalt'}},
 {'name': '34R',
  'length': {'ft': 15912, 'm': 4850},
  'surface': {'code': 'ASPH', 'name': 'Asphalt'}}]

In [92]:
# Found significant data and store in variable flight_data
flight_data = dict1['schedule']['departures']['data']
flight_data

[{'flight': {'identification': {'id': None,
    'row': 5341427706,
    'number': {'default': 'QR8434', 'alternative': None},
    'callsign': None,
    'codeshare': None},
   'status': {'live': False,
    'text': 'Scheduled',
    'icon': None,
    'estimated': None,
    'ambiguous': False,
    'generic': {'status': {'text': 'scheduled',
      'type': 'departure',
      'color': 'gray',
      'diverted': None},
     'eventTime': {'utc': None, 'local': None}}},
   'aircraft': {'model': {'code': '77X', 'text': ''},
    'hex': '',
    'registration': '',
    'serialNo': '',
    'images': None},
   'owner': None,
   'airline': {'name': 'Qatar Airways',
    'code': {'iata': 'QR', 'icao': 'QTR'},
    'short': 'Qatar Airways'},
   'airport': {'origin': {'timezone': {'name': 'Asia/Qatar',
      'offset': 10800,
      'abbr': '+03',
      'abbrName': None,
      'isDst': False},
     'info': {'terminal': None, 'baggage': None, 'gate': None}},
    'destination': {'code': {'iata': 'BKK', 'icao': 'V

In [93]:
# convert nested dictionary into a flat dictionary
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
l = []
# append to new list, covert to pandas dataframe
for i in flight_data:
    l.append(flatten(i))
df = pd.DataFrame(l)
df

,flight_identification_id,flight_identification_row,flight_identification_number_default,flight_identification_number_alternative,flight_identification_callsign,flight_identification_codeshare,flight_status_live,flight_status_text,flight_status_icon,flight_status_estimated,...,flight_aircraft_country_alpha2,flight_aircraft_country_alpha3,flight_aircraft_restricted,flight_aircraft_age_availability,flight_aircraft_availability_serialNo,flight_aircraft_availability_age,flight_owner_name,flight_owner_code_iata,flight_owner_code_icao,flight_owner_logo
0,None,5341427706,QR8434,None,None,None,False,Scheduled,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,5343637775,QR8360,9T8360,None,None,False,Scheduled,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,5344268274,QR1166,None,None,"[VA6208, WB1178, WY6470, AA8204, BA2327, CX921...",False,Scheduled,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,5341427024,QR1190,None,None,"[WY6482, MH4148, CX9308, IB902, BA2360, AA8263...",False,Estimated dep 04:40,green,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,5343637776,QR8111,None,QTR8111,None,False,Estimated dep 04:55,green,None,...,QA,QAT,False,True,True,True,Qatar Airways,QR,QTR,s3:QR_QTR.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,None,5344274334,QR634,None,None,None,False,Scheduled,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,None,5341425787,QR183,None,QTR53L,None,False,Estimated dep 09:45,green,None,...,QA,QAT,False,True,True,True,Qatar Airways,QR,QTR,s3:QR_QTR.png
97,None,5341376983,G9136,None,None,None,False,Estimated dep 09:55,green,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,None,5341366446,FZ2,None,None,None,False,Estimated dep 10:00,green,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning

In [94]:
# Check values for cleaning
df.count()

flight_identification_id                      0
flight_identification_row                   100
flight_identification_number_default        100
flight_identification_number_alternative      3
flight_identification_callsign               45
                                           ... 
flight_aircraft_availability_age             38
flight_owner_name                            38
flight_owner_code_iata                       38
flight_owner_code_icao                       38
flight_owner_logo                            34
Length: 75, dtype: int64

In [95]:
# extract rows which are null
x = df.isnull().sum()
null_list = []
for index, value in x.items():
    if value == 100:
        null_list.append(index)
        print(index,value)

flight_identification_id 100
flight_status_estimated 100
flight_status_generic_status_diverted 100
flight_aircraft_images 100
flight_owner 100
flight_airport_origin_timezone_abbrName 100
flight_airport_origin_info_baggage 100
flight_airport_real 100
flight_time_real_departure 100
flight_time_real_arrival 100
flight_time_estimated_arrival 100
flight_time_other_eta 100
flight_time_other_duration 100


In [96]:
# Aggregate list of null columns to be used in drop
null_list

['flight_identification_id',
 'flight_status_estimated',
 'flight_status_generic_status_diverted',
 'flight_aircraft_images',
 'flight_owner',
 'flight_airport_origin_timezone_abbrName',
 'flight_airport_origin_info_baggage',
 'flight_airport_real',
 'flight_time_real_departure',
 'flight_time_real_arrival',
 'flight_time_estimated_arrival',
 'flight_time_other_eta',
 'flight_time_other_duration']

In [97]:
df = df.drop(columns=['flight_identification_id',
 'flight_status_estimated',
 'flight_status_generic_status_diverted',
 'flight_aircraft_images',
 'flight_owner',
 'flight_airport_origin_timezone_abbrName',
 'flight_airport_origin_info_baggage',
 'flight_airport_real',
 'flight_time_real_departure',
 'flight_time_real_arrival',
 'flight_time_estimated_arrival',
 'flight_time_other_eta',
 'flight_time_other_duration'])

In [98]:
df.head()

,flight_identification_row,flight_identification_number_default,flight_identification_number_alternative,flight_identification_callsign,flight_identification_codeshare,flight_status_live,flight_status_text,flight_status_icon,flight_status_ambiguous,flight_status_generic_status_text,...,flight_aircraft_country_alpha2,flight_aircraft_country_alpha3,flight_aircraft_restricted,flight_aircraft_age_availability,flight_aircraft_availability_serialNo,flight_aircraft_availability_age,flight_owner_name,flight_owner_code_iata,flight_owner_code_icao,flight_owner_logo
0,5341427706,QR8434,None,None,None,False,Scheduled,None,False,scheduled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5343637775,QR8360,9T8360,None,None,False,Scheduled,None,False,scheduled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5344268274,QR1166,None,None,"[VA6208, WB1178, WY6470, AA8204, BA2327, CX921...",False,Scheduled,None,False,scheduled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5341427024,QR1190,None,None,"[WY6482, MH4148, CX9308, IB902, BA2360, AA8263...",False,Estimated dep 04:40,green,False,estimated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5343637776,QR8111,None,QTR8111,None,False,Estimated dep 04:55,green,False,estimated,...,QA,QAT,False,True,True,True,Qatar Airways,QR,QTR,s3:QR_QTR.png


In [99]:
# Here there are more values which aren't completely null but still provide no use
df.isnull().sum()

flight_identification_row                    0
flight_identification_number_default         0
flight_identification_number_alternative    97
flight_identification_callsign              55
flight_identification_codeshare             77
                                            ..
flight_aircraft_availability_age            62
flight_owner_name                           62
flight_owner_code_iata                      62
flight_owner_code_icao                      62
flight_owner_logo                           66
Length: 62, dtype: int64

In [100]:
# Decided to get list of columns to instead pull neccessary information rather than delete unncessary data
columns_list = df.columns.tolist()
columns_list

['flight_identification_row',
 'flight_identification_number_default',
 'flight_identification_number_alternative',
 'flight_identification_callsign',
 'flight_identification_codeshare',
 'flight_status_live',
 'flight_status_text',
 'flight_status_icon',
 'flight_status_ambiguous',
 'flight_status_generic_status_text',
 'flight_status_generic_status_type',
 'flight_status_generic_status_color',
 'flight_status_generic_eventTime_utc',
 'flight_status_generic_eventTime_local',
 'flight_aircraft_model_code',
 'flight_aircraft_model_text',
 'flight_aircraft_hex',
 'flight_aircraft_registration',
 'flight_aircraft_serialNo',
 'flight_airline_name',
 'flight_airline_code_iata',
 'flight_airline_code_icao',
 'flight_airline_short',
 'flight_airport_origin_timezone_name',
 'flight_airport_origin_timezone_offset',
 'flight_airport_origin_timezone_abbr',
 'flight_airport_origin_timezone_isDst',
 'flight_airport_origin_info_terminal',
 'flight_airport_origin_info_gate',
 'flight_airport_destinat

In [101]:
# Create dataframe with desired columns
df_new = df.loc[:,['flight_airline_code_icao', 
                   'flight_identification_number_default',
                   'flight_status_text',
                   'flight_airport_destination_timezone_name',
                   'flight_airport_destination_timezone_abbr',
                   'flight_airport_destination_name',
                   'flight_airport_destination_position_country_name',
                   'flight_airport_destination_position_region_city',
                   'flight_time_scheduled_departure',
                   'flight_time_scheduled_arrival',
                  ]]
df_new.head()

,flight_airline_code_icao,flight_identification_number_default,flight_status_text,flight_airport_destination_timezone_name,flight_airport_destination_timezone_abbr,flight_airport_destination_name,flight_airport_destination_position_country_name,flight_airport_destination_position_region_city,flight_time_scheduled_departure,flight_time_scheduled_arrival
0,QTR,QR8434,Scheduled,Asia/Bangkok,+07,Bangkok Suvarnabhumi Airport,Thailand,Bangkok,1680138300,1680161100
1,QTR,QR8360,Scheduled,Asia/Hong_Kong,HKT,Hong Kong International Airport,Hong Kong,Hong Kong,1680138600,1680168600
2,QTR,QR1166,Scheduled,Asia/Riyadh,+03,Riyadh King Khalid International Airport,Saudi Arabia,Riyadh,1680139800,1680145200
3,QTR,QR1190,Estimated dep 04:40,Asia/Riyadh,+03,Jeddah King Abdulaziz International Airport,Saudi Arabia,Jeddah,1680140400,1680149700
4,QTR,QR8111,Estimated dep 04:55,Europe/Rome,CEST,Milan Malpensa Airport,Italy,Milan,1680140700,1680163200


In [102]:
# View destination timezones needed for time conversions
print(df_new['flight_airport_destination_timezone_name'].unique())

['Asia/Bangkok' 'Asia/Hong_Kong' 'Asia/Riyadh' 'Europe/Rome'
 'Asia/Kolkata' 'Europe/Amsterdam' 'Europe/Istanbul' 'Asia/Kuwait'
 'Asia/Amman' 'America/Sao_Paulo' 'Europe/Paris' 'Europe/Oslo'
 'Europe/Athens' 'Asia/Tehran' 'Asia/Dubai' 'Asia/Beirut'
 'America/Montreal' 'Africa/Mogadishu' 'Africa/Dar_es_Salaam'
 'Europe/Madrid' 'America/Chicago' 'America/Los_Angeles'
 'America/New_York' 'Asia/Manila' 'Africa/Lagos' 'Asia/Muscat'
 'Europe/London' 'Asia/Makassar' 'Europe/Dublin' 'Europe/Berlin'
 'Asia/Nicosia' 'Europe/Luxembourg' 'Europe/Copenhagen'
 'Asia/Kuala_Lumpur' 'Europe/Prague' 'Africa/Johannesburg' 'Asia/Dhaka'
 'Europe/Helsinki' 'Europe/Stockholm' 'Europe/Zurich' 'Europe/Warsaw'
 'Europe/Bucharest' 'Asia/Singapore' 'Africa/Cairo' 'Europe/Budapest'
 'Asia/Jakarta' 'Asia/Baghdad' 'Africa/Kampala' 'Asia/Colombo'
 'Africa/Tunis' 'Asia/Seoul' 'Asia/Baku' 'Indian/Maldives'
 'Asia/Kathmandu' 'Europe/Vienna']


In [103]:
# Convert the 'flight_time_scheduled_departure' column from Unix time to a datetime object
df_new['flight_time_scheduled_departure'] = pd.to_datetime(df_new['flight_time_scheduled_departure'], unit='s')

# Create a new column 'scheduled_departure_local_time' to store the scheduled departure time in the destination airport's timezone
df_new['scheduled_departure_local_time'] = None

# Iterate through the unique timezones in the 'flight_airport_destination_timezone_name' column
for timezone in df_new['flight_airport_destination_timezone_name'].unique():
    # Use the pandas tz_localize() function to set the timezone of the 'scheduled_departure_local_time' column to the destination airport's timezone
    df_new.loc[df_new['flight_airport_destination_timezone_name'] == timezone, 'scheduled_departure_local_time'] = pd.to_datetime(df_new.loc[df_new['flight_airport_destination_timezone_name'] == timezone, 'flight_time_scheduled_departure'], unit='s').dt.tz_localize(timezone)

df_new.head()

,flight_airline_code_icao,flight_identification_number_default,flight_status_text,flight_airport_destination_timezone_name,flight_airport_destination_timezone_abbr,flight_airport_destination_name,flight_airport_destination_position_country_name,flight_airport_destination_position_region_city,flight_time_scheduled_departure,flight_time_scheduled_arrival,scheduled_departure_local_time
0,QTR,QR8434,Scheduled,Asia/Bangkok,+07,Bangkok Suvarnabhumi Airport,Thailand,Bangkok,2023-03-30 01:05:00,1680161100,2023-03-30 01:05:00+07:00
1,QTR,QR8360,Scheduled,Asia/Hong_Kong,HKT,Hong Kong International Airport,Hong Kong,Hong Kong,2023-03-30 01:10:00,1680168600,2023-03-30 01:10:00+08:00
2,QTR,QR1166,Scheduled,Asia/Riyadh,+03,Riyadh King Khalid International Airport,Saudi Arabia,Riyadh,2023-03-30 01:30:00,1680145200,2023-03-30 01:30:00+03:00
3,QTR,QR1190,Estimated dep 04:40,Asia/Riyadh,+03,Jeddah King Abdulaziz International Airport,Saudi Arabia,Jeddah,2023-03-30 01:40:00,1680149700,2023-03-30 01:40:00+03:00
4,QTR,QR8111,Estimated dep 04:55,Europe/Rome,CEST,Milan Malpensa Airport,Italy,Milan,2023-03-30 01:45:00,1680163200,2023-03-30 01:45:00+02:00


In [104]:
# Repeat for scheduled_arrival
df_new['flight_time_scheduled_arrival'] = pd.to_datetime(df_new['flight_time_scheduled_arrival'], unit='s')

df_new['scheduled_arrival_local_time'] = None

for timezone in df_new['flight_airport_destination_timezone_name'].unique():

    df_new.loc[df_new['flight_airport_destination_timezone_name'] == timezone, 'scheduled_arrival_local_time'] = pd.to_datetime(df_new.loc[df_new['flight_airport_destination_timezone_name'] == timezone, 'flight_time_scheduled_arrival'], unit='s').dt.tz_localize(timezone)

df_new.head()

,flight_airline_code_icao,flight_identification_number_default,flight_status_text,flight_airport_destination_timezone_name,flight_airport_destination_timezone_abbr,flight_airport_destination_name,flight_airport_destination_position_country_name,flight_airport_destination_position_region_city,flight_time_scheduled_departure,flight_time_scheduled_arrival,scheduled_departure_local_time,scheduled_arrival_local_time
0,QTR,QR8434,Scheduled,Asia/Bangkok,+07,Bangkok Suvarnabhumi Airport,Thailand,Bangkok,2023-03-30 01:05:00,2023-03-30 07:25:00,2023-03-30 01:05:00+07:00,2023-03-30 07:25:00+07:00
1,QTR,QR8360,Scheduled,Asia/Hong_Kong,HKT,Hong Kong International Airport,Hong Kong,Hong Kong,2023-03-30 01:10:00,2023-03-30 09:30:00,2023-03-30 01:10:00+08:00,2023-03-30 09:30:00+08:00
2,QTR,QR1166,Scheduled,Asia/Riyadh,+03,Riyadh King Khalid International Airport,Saudi Arabia,Riyadh,2023-03-30 01:30:00,2023-03-30 03:00:00,2023-03-30 01:30:00+03:00,2023-03-30 03:00:00+03:00
3,QTR,QR1190,Estimated dep 04:40,Asia/Riyadh,+03,Jeddah King Abdulaziz International Airport,Saudi Arabia,Jeddah,2023-03-30 01:40:00,2023-03-30 04:15:00,2023-03-30 01:40:00+03:00,2023-03-30 04:15:00+03:00
4,QTR,QR8111,Estimated dep 04:55,Europe/Rome,CEST,Milan Malpensa Airport,Italy,Milan,2023-03-30 01:45:00,2023-03-30 08:00:00,2023-03-30 01:45:00+02:00,2023-03-30 08:00:00+02:00


In [105]:
df_new = df_new.rename(columns={'flight_airline_code_icao': 'ICAO', 'flight_identification_number_default':'Flight ID'
                                ,'flight_status_text':'Flight_Status','flight_airport_destination_timezone_name':'Destination Timezone'
                                ,'flight_airport_destination_name': 'Destination Airport','flight_airport_destination_position_country_name':'Destination Country'
                                ,'flight_airport_destination_position_region_city':'Destination City'
                                , 'flight_time_scheduled_departure': 'Scheduled Departure (UTC)'
                                , 'flight_time_scheduled_arrival': 'Scheduled Arrival (UTC)'
                                , 'scheduled_departure_local_time': 'Schedule Departure Local Time'
                                , 'scheduled_arrival_local_time': 'Schedule Arrival Local Time'})

df_new.head()

,ICAO,Flight ID,Flight_Status,Destination Timezone,flight_airport_destination_timezone_abbr,Destination Airport,Destination Country,Destination City,Scheduled Departure (UTC),Scheduled Arrival (UTC),Schedule Departure Local Time,Schedule Arrival Local Time
0,QTR,QR8434,Scheduled,Asia/Bangkok,+07,Bangkok Suvarnabhumi Airport,Thailand,Bangkok,2023-03-30 01:05:00,2023-03-30 07:25:00,2023-03-30 01:05:00+07:00,2023-03-30 07:25:00+07:00
1,QTR,QR8360,Scheduled,Asia/Hong_Kong,HKT,Hong Kong International Airport,Hong Kong,Hong Kong,2023-03-30 01:10:00,2023-03-30 09:30:00,2023-03-30 01:10:00+08:00,2023-03-30 09:30:00+08:00
2,QTR,QR1166,Scheduled,Asia/Riyadh,+03,Riyadh King Khalid International Airport,Saudi Arabia,Riyadh,2023-03-30 01:30:00,2023-03-30 03:00:00,2023-03-30 01:30:00+03:00,2023-03-30 03:00:00+03:00
3,QTR,QR1190,Estimated dep 04:40,Asia/Riyadh,+03,Jeddah King Abdulaziz International Airport,Saudi Arabia,Jeddah,2023-03-30 01:40:00,2023-03-30 04:15:00,2023-03-30 01:40:00+03:00,2023-03-30 04:15:00+03:00
4,QTR,QR8111,Estimated dep 04:55,Europe/Rome,CEST,Milan Malpensa Airport,Italy,Milan,2023-03-30 01:45:00,2023-03-30 08:00:00,2023-03-30 01:45:00+02:00,2023-03-30 08:00:00+02:00


## Import new dataset to be used for connection

In [106]:
# read new dataset to connect ICAO Codes
df_airline = pd.read_csv('airlines.csv')
df_airline.head()

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N


In [107]:
# Clean read-in data
df_airline.drop('Airline ID',axis = 1, inplace=True)
df_airline.drop('Alias',axis = 1, inplace=True)
df_airline.drop('IATA',axis = 1, inplace=True)
df_airline.head()

,Name,ICAO,Callsign,Country,Active
0,Unknown,NaN,\N,\N,Y
1,Private flight,NaN,NaN,NaN,Y
2,135 Airways,GNL,GENERAL,United States,N
3,1Time Airline,RNX,NEXTIME,South Africa,Y
4,2 Sqn No 1 Elementary Flying Training School,WYT,NaN,United Kingdom,N


In [108]:
# merged/join connected data frames to show connection
merged_df = pd.merge(df_new, df_airline, on ='ICAO')
merged_df.head()

,ICAO,Flight ID,Flight_Status,Destination Timezone,flight_airport_destination_timezone_abbr,Destination Airport,Destination Country,Destination City,Scheduled Departure (UTC),Scheduled Arrival (UTC),Schedule Departure Local Time,Schedule Arrival Local Time,Name,Callsign,Country,Active
0,QTR,QR8434,Scheduled,Asia/Bangkok,+07,Bangkok Suvarnabhumi Airport,Thailand,Bangkok,2023-03-30 01:05:00,2023-03-30 07:25:00,2023-03-30 01:05:00+07:00,2023-03-30 07:25:00+07:00,Qatar Airways,QATARI,Qatar,Y
1,QTR,QR8360,Scheduled,Asia/Hong_Kong,HKT,Hong Kong International Airport,Hong Kong,Hong Kong,2023-03-30 01:10:00,2023-03-30 09:30:00,2023-03-30 01:10:00+08:00,2023-03-30 09:30:00+08:00,Qatar Airways,QATARI,Qatar,Y
2,QTR,QR1166,Scheduled,Asia/Riyadh,+03,Riyadh King Khalid International Airport,Saudi Arabia,Riyadh,2023-03-30 01:30:00,2023-03-30 03:00:00,2023-03-30 01:30:00+03:00,2023-03-30 03:00:00+03:00,Qatar Airways,QATARI,Qatar,Y
3,QTR,QR1190,Estimated dep 04:40,Asia/Riyadh,+03,Jeddah King Abdulaziz International Airport,Saudi Arabia,Jeddah,2023-03-30 01:40:00,2023-03-30 04:15:00,2023-03-30 01:40:00+03:00,2023-03-30 04:15:00+03:00,Qatar Airways,QATARI,Qatar,Y
4,QTR,QR8111,Estimated dep 04:55,Europe/Rome,CEST,Milan Malpensa Airport,Italy,Milan,2023-03-30 01:45:00,2023-03-30 08:00:00,2023-03-30 01:45:00+02:00,2023-03-30 08:00:00+02:00,Qatar Airways,QATARI,Qatar,Y


In [109]:
# send to flat file
df_new.to_csv('flightinfo.csv')
df_airline.to_csv('airlineinfo.csv')

## Conclusion
In summary, building such a pipeline has displayed the power and efficiency that can be pursued of API Calling even in a personal project. Although REST APIs in particular seem to be quite static and hard to use from the outside, working with them directly and with guidance along the way has shown the overall power especially due to the natural automation of the pipeline. Despite the fact that there is room for improvement, I feel that this is a good starting point for the building of data workflows within an API.